In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# **Data Importing**

In [2]:
data = pd.read_csv('train.csv')
data.shape
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# **Data Description**

In [3]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# **Data Cleaning**

In [5]:
# Dropping Duplicates
data = data.drop_duplicates()
data.shape

(891, 12)

In [6]:
# Checking null values
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

* **There are 177 null values in Age column**
* **2 null values in Embarked column**

In [7]:
from sklearn.impute import SimpleImputer
median_imp = SimpleImputer(strategy="median")
mode_imp = SimpleImputer(strategy="most_frequent")
const_imp = SimpleImputer(strategy="constant",fill_value=0)

In [8]:
# Fill missing values in embarked with mode
data['Embarked'] = mode_imp.fit_transform(data[['Embarked']])
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [9]:
# Fill missing values in Age column with median
data['Age'] = median_imp.fit_transform(data[['Age']])

In [10]:
print(data['Age'].median())
data.isna().sum()

28.0


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [11]:
data['Fare'] = const_imp.fit_transform(data[['Fare']])

In [12]:
import seaborn as sns
import warnings; warnings.filterwarnings("ignore")

In [13]:
data.shape
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

## Feature Embedding


In [14]:
# Segmenting Sex column as per Age, Age less than 18 as Child, Age greater than 18 as Males and Females as per their gender.
data['GenderClass'] = data.apply(lambda x: 'child' if x['Age'] < 18 else x['Sex'],axis=1)

In [15]:
# Creating a new feature FamilySize from Sibsp and Parch
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [16]:
# Dropping unnecessary columns like Name, PassengeId, Ticket and Cabin because of too many missing values.
# Also drop gender column
data = data.drop(labels=['PassengerId','Name','Ticket','Cabin','SibSp','Parch','Sex'],axis=1)

In [17]:
data.head()

,Survived,Pclass,Age,Fare,Embarked,GenderClass,FamilySize
0,0,3,22.0,7.2500,S,male,2
1,1,1,38.0,71.2833,C,female,2
2,1,3,26.0,7.9250,S,female,1
3,1,1,35.0,53.1000,S,female,2
4,0,3,35.0,8.0500,S,male,1


In [18]:
# Data is Clean

# **Data Preprocessing**

In [19]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [20]:
# Dummification of GenderClass & Embarked.
data = pd.get_dummies(data, columns=['GenderClass','Embarked'], drop_first=True)

In [21]:
# mm_scale = MinMaxScaler()
# data['Age'] = mm_scale.fit_transform(data[['Age']])

In [22]:
# r_scale = RobustScaler()
# data['Fare'] = r_scale.fit_transform(data[['Fare']])

In [23]:
data.head()

,Survived,Pclass,Age,Fare,FamilySize,GenderClass_female,GenderClass_male,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,2,0,1,0,1
1,1,1,38.0,71.2833,2,1,0,0,0
2,1,3,26.0,7.9250,1,1,0,0,1
3,1,1,35.0,53.1000,2,1,0,0,1
4,0,3,35.0,8.0500,1,0,1,0,1


**Preparing X and Y data**

In [24]:
X = data.loc[:,data.columns != 'Survived']

In [25]:
y = data.Survived

**Train-test Splitting**

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=78)

# **Logistic Regression**

In [27]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression()

In [28]:
y_pred_train = logreg.predict(X_train)
y_pred_test = logreg.predict(X_test)  

# **Model Evaluation**

In [29]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [30]:
acc_train = accuracy_score(y_train, y_pred_train)
print(acc_train)
acc_test = accuracy_score(y_test, y_pred_test)
print(acc_test)

0.797752808988764
0.8491620111731844


In [31]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.91      0.85      0.88       116
           1       0.76      0.84      0.80        63

    accuracy                           0.85       179
   macro avg       0.83      0.85      0.84       179
weighted avg       0.86      0.85      0.85       179



In [32]:
confusion_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred_test))
confusion_matrix

,0,1
0,99,17
1,10,53


In [33]:
test = pd.read_csv('test.csv')
df = test
df.shape

(418, 11)

In [34]:
df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [35]:
df['Age'] = median_imp.transform(df[['Age']])

In [36]:
df['Fare'] = const_imp.transform(df[['Fare']])

In [37]:
df['GenderClass'] = df.apply(lambda x: 'child' if x['Age'] < 18 else x['Sex'],axis=1)

In [38]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [39]:
df = df.drop(labels=['PassengerId','Name','Ticket','Cabin','SibSp','Parch','Sex'],axis=1)

In [40]:
df = pd.get_dummies(df, columns=['GenderClass','Embarked'], drop_first=True)

In [41]:
df_pred = logreg.predict(df)

In [42]:
output = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':df_pred})

In [43]:
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
